In [128]:
import os 
import json 
import re
import numpy as np

from collections import Counter, OrderedDict

np.set_printoptions(precision=4)

# Extract utilities

In [96]:
bound = 0.01
# Go from bid -> utility
def get_utility(bid, pref):
    util = 0
    issuelist = list(pref.keys())
    for issueindex, issue in enumerate(issuelist):
        # values of issue in given bid
        value = pref[issue][bid[issueindex]]
        # weight of issue in given bid
        weight = pref[issue]['weight']
        util += weight*value
    return util

# Utility change to discrete move type
def delta_mapping(delta_util):
    if (abs(delta_util[0]) <= bound and abs(delta_util[1]) <= bound):
        return 'silent'
    if (abs(delta_util[0]) <= bound and delta_util[1]) > bound:
        return 'nice'
    if (delta_util[0] <= 0 and delta_util[1] <= 0):
        return 'unfortunate'
    if (delta_util[0] > 0 and delta_util[1] <= 0):
        return 'selfish'
    if (delta_util[0] > 0 and delta_util[1] >= 0):
        return 'fortunate'
    if (delta_util[0] < 0 and delta_util[1] > 0):
        return 'concession'
    print(delta_util)

# List of bids to list of discrete moves
def discritized_mapping(agent_bids):
    mapped_utils_discrete = []
    prev_utils = agent_bids[0]
    for new_utils in agent_bids[1:]:
        delta_util1 = new_utils[0] - prev_utils[0]
        delta_util2 = new_utils[1] - prev_utils[1]
        delta_util = (delta_util1, delta_util2)
        mapped_utils_discrete.append(delta_mapping(delta_util))
        prev_utils = new_utils
    return mapped_utils_discrete
        
    
def retrieve_all_agents_bids(train):
    # Useful structures
    all_issues = train['issues']
    pref1 = train['Utility1']
    pref2 = train['Utility2']
    all_bids = train['bids']

    mapped_utils_a1 = []
    mapped_utils_a2 = []

    # Parse utility values of bids
    for bid in all_bids:
        r = bid['round']
#         print(bid)
        # stop if the negotiation session has ended
        if 'agent1' in bid:
            bid_agent1 = bid['agent1'].split(',')
            u1_b1 = get_utility(bid_agent1, pref1)
            u2_b1 = get_utility(bid_agent1, pref2)
            # Save the bid -> utility mapping
            mapped_utils_a1.append([u1_b1, u2_b1, int(r)])
        if 'agent2' in bid:
            bid_agent2 = bid['agent2'].split(',')
            u1_b2 = get_utility(bid_agent2, pref1)
            u2_b2 = get_utility(bid_agent2, pref2)
            mapped_utils_a2.append([u2_b2, u1_b2, int(r)])


    agent1_bids = [mapped_utils_a1[i][0:2] for i in range(len(mapped_utils_a1))]
    agent2_bids = [mapped_utils_a2[i][0:2] for i in range(len(mapped_utils_a2))]

    # Discritize bidspace
    agent1_bids_discrete = discritized_mapping(agent1_bids)
    agent2_bids_discrete = discritized_mapping(agent2_bids)
    
    return (agent1_bids_discrete, agent2_bids_discrete)

In [97]:
path='train/'

train_files = os.listdir(path)
agent_count_mapping = {}
for t in train_files:
    train = json.load(open(os.path.join(path, t)))
    a1_name, a2_name = re.split(r'[^A-Za-z]+', t.strip('.json'))[0:2]
    a1_bids, a2_bids = retrieve_all_agents_bids(train)
    if a1_name not in agent_count_mapping:
        agent_count_mapping[a1_name] = []
    if a2_name not in agent_count_mapping:
        agent_count_mapping[a2_name] = []
    agent_count_mapping[a1_name] = agent_count_mapping[a1_name] + a1_bids
    agent_count_mapping[a2_name] = agent_count_mapping[a2_name] + a2_bids

# Sensor model

In [152]:
sensor_model = {}
for k, v in agent_count_mapping.items():
    cnt_bids = Counter(v)
    total = len(v)
    for key in cnt_bids:
        cnt_bids[key] /= total
    sensor_model[k] = dict(cnt_bids)
    
possible_moves = ['silent', 'concession', 'unfortunate', 'nice', 'fortunate', 'selfish']

for k, moves in sensor_model.items():
    for pm in possible_moves:
        if pm not in moves:
            moves[pm] = 0.0
    sensor_model[k] = np.array(list(dict(OrderedDict(sorted(moves.items()))).values()))

print(sorted(possible_moves))
sensor_model

['concession', 'fortunate', 'nice', 'selfish', 'silent', 'unfortunate']


{'hardheaded': array([0.0382, 0.    , 0.0172, 0.    , 0.9008, 0.0439]),
 'random': array([0.4169, 0.1118, 0.0091, 0.3384, 0.003 , 0.1208]),
 'conceder': array([0.0806, 0.    , 0.2246, 0.    , 0.402 , 0.2928]),
 'tft': array([0.0961, 0.0146, 0.0771, 0.0306, 0.6215, 0.1601])}